# Create an LLM Agent Capable of Math and Machine Learning Operations, Using the OpenAI API.

You are an AI engineer working at a financial services firm. Your financial analysts want to use modern large language models as a tool, but need access to mathematical and regression tools that LLM’s are not natively skilled at. Your manager has asked you to prototype an LLM agent solution, coding directly against the OpenAI chat completions API, that augments a chatbot with local tools capable of performing basic mathematical operations and linear regression. Your LLM agent will be evaluated by its ability to successfully compute math and regression problems given as natural language prompts.

# Write a function to serve a math tool

In [1]:
import json

def perform_math_operation(operation, args=[]):
    result = 0
    if operation == 'add':
        result = sum(args)
    elif operation == 'subtract':
        result = args[0] - sum(args[1:])
    elif operation == 'multiply':
        result = 1
        for num in args:
            result *= num
    elif operation == 'divide':
        result = args[0]
        for num in args[1:]:
            result /= num
    else:
        raise ValueError("Unsupported operation")
 
    retval = {"Result": result}
    return json.dumps(retval)

In [2]:
perform_math_operation("multiply", [8,2,2])

'{"Result": 32}'

# Write a function to serve as a Linear Regression Tool

Implement a Python function that may be used by an LLM agent to predict the next value in a series of numbers, using linear regression.  This function will accept a single argument that is a list of numeric values in a series, and will return the predicted next value in JSON format, labeled as Next value.

In [7]:
import numpy as np
from sklearn.linear_model import LinearRegression
 
def predict_next_in_series(series):
    x = np.array(range(len(series))).reshape(-1, 1)
    y = np.array(series).reshape(-1, 1)
    model = LinearRegression().fit(x, y)
    next_index = len(series)
    next_value = model.predict([[next_index]])
    next_info = {
        "Next value": next_value[0][0]
    }
    return json.dumps(next_info)

In [8]:
predict_next_in_series([2, 4, 6, 8])

'{"Next value": 10.0}'

# Create JSON Schema Objects For Our Tools

Our math and regression tools must be defined for the OpenAI Chat Completions API, in their documented format that includes a JSON Schema definition for the properties of the arguments our tools accept (under parameters,) a name for each function (under name,) and a description of the circumstances under which this tool should be used (under description.) Create definitions for both the mathematical operations tool and linear regression tool functions, and combine them both into a list named tools.

In [9]:
mathTool = {
        "type": "function",
        "function": {
            "name": "perform_math_operation",
            "description": "Perform a mathematical operation",
            "parameters": {
                "type": "object",
                "properties": {
                    "operation": 
                    {
                        "type": "string", 
                        "enum": ["add", "subtract", "multiply", "divide"],
                        "description": "The requested mathematical operation to perform."
                    },
                    "operands": 
                    {
                        "type": "array", 
                        "items": {"type": "number"},
                        "description": "The operands of the requested mathematical operation, from left to right."
                    }
                },
                "required": ["operation", "operands"]
            },
        }
    }

regressionTool = {
        "type": "function",
        "function": {
            "name": "predict_next_in_series",
            "description": "Predict the next number in a series using linear regression",
            "parameters": {
                "type": "object",
                "properties": {
                    "series": {
                        "type": "array", 
                        "items": {"type": "number"},
                        "description": "The list of numbers in the series to be completed."
                    }
                },
                "required": ["series"]
            },
        }
    }

In [11]:
tools = [mathTool, regressionTool]
# tools

# Invoke OpenAI chat Completions API with Tools

Use the OpenAI Chat Completions API to answer a given prompt as a parameter to a function named llm_agent, given a pre-defined list of tools it may use at its discretion for performing math operations, or predicting the next value in a series. If the chat completions API call returns one or more requested tool calls, invoke the appropriate function(s) to execute the tool(s), append their result to the chat messages, and call the Chat Completions API again for a final response including the tool output. Use the gpt-3.5-turbo-0613 model. Incorporate print calls indicating when each step is executed to be used for debugging and validation purposes.

In [13]:
import openai
from openai import OpenAI
 
client = OpenAI()

In [21]:
# function to dispatch an initial chat completions request together with the available tools
def make_initial_request(messages):
 
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        tools=tools,
        tool_choice=None
    )
    
    response_message = response.choices[0].message
    
    # print('response_message: ',response_message)
    
    return response_message

In [26]:
# function to invoke one or more defined tool functions, and return a response in OpenAI's required format
def call_tool(tool):
    
    # print("calling tool: ")
    # print(tool)
    
    response = {}
    
    function_name = tool.function.name
    if (function_name == "perform_math_operation"):
        args = json.loads(tool.function.arguments)
        fn_response = perform_math_operation(operation=args.get("operation"),args=args.get("operands"))
        response = {
            "tool_call_id": tool.id,
            "role": "tool",
            "name": function_name,
            "content": fn_response,
        }
    elif (function_name == "predict_next_in_series"):
        args = json.loads(tool.function.arguments)
        fn_response = predict_next_in_series(series=args.get("series"))
        response = {
            "tool_call_id": tool.id,
            "role": "tool",
            "name": function_name,
            "content": fn_response,
        }
    return response

In [31]:
# function to accept a given prompt, call any requested tools from the chat completions API, append their result(s) to the message chain, and perform a second chat completions call if necessary:

def llm_agent(prompt):
    messages = [{"role": "user", "content": prompt}]
    
    first_response = make_initial_request(messages)
    
    tool_calls = first_response.tool_calls
    if (tool_calls):
 
        messages.append(first_response)
        
        for tool in tool_calls:
            tool_response = call_tool(tool)
            messages.append(tool_response)
 
        # print("second call: ")
        # print(messages)
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages = messages,
        )
            
        return second_response
    else:
        return response

# Test the resulting LLM Agent

In [28]:
prompt = "What is 5 divided by 3?"
print (llm_agent(prompt))

second call: 
ChatCompletion(id=None, choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='5 divided by 3 is approximately 1.67', role='assistant', function_call=None, tool_calls=[], name=''))], created=None, model=None, object=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=64, total_tokens=75), failure_reason='', status='STATUS_SUCCESS')


In [30]:
prompt = "What is the next value in the series 1, 3, 5, 7, 11?"
print (llm_agent(prompt))

ChatCompletion(id=None, choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The next value in the series 1, 3, 5, 7, 11 is 12.6 (rounded to one decimal place).', role='assistant', function_call=None, tool_calls=[], name=''))], created=None, model=None, object=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=32, prompt_tokens=81, total_tokens=113), failure_reason='', status='STATUS_SUCCESS')
